In [17]:

import math
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data
import util

from args_QANet import get_train_args

from collections import OrderedDict
from json import dumps

from models_QANet import QANet

from tensorboardX import SummaryWriter
from tqdm import tqdm
from ujson import load as json_load
from util import collate_fn, SQuAD, get_available_devices


char_vocab_size = 1376 # 节省内存
word_vectors = util.torch_from_json("data/word_emb.json")
model = QANet(
    word_vectors=word_vectors,
    char_vocab_size=char_vocab_size,
    char_dim=200,
    d_model=128,
    drop_prob=0.1,
    num_mod_blocks=5,  # 节省内存=============================
    maximum_context_length=400)
# ==============================================================================


In [10]:
for name, parms in model.named_parameters():
	    print('-->name:', name, '-->grad_requirs:', parms.requires_grad)

-->name: emb.word_emb.weight -->grad_requirs: False
-->name: emb.char_emb.weight -->grad_requirs: True
-->name: emb.hwy.transforms.0.weight -->grad_requirs: True
-->name: emb.hwy.transforms.0.bias -->grad_requirs: True
-->name: emb.hwy.transforms.1.weight -->grad_requirs: True
-->name: emb.hwy.transforms.1.bias -->grad_requirs: True
-->name: emb.hwy.gates.0.weight -->grad_requirs: True
-->name: emb.hwy.gates.0.bias -->grad_requirs: True
-->name: emb.hwy.gates.1.weight -->grad_requirs: True
-->name: emb.hwy.gates.1.bias -->grad_requirs: True
-->name: emb_proj.weight -->grad_requirs: True
-->name: emb_proj.bias -->grad_requirs: True
-->name: emb_enc.enc.conv_layers.0.depth.weight -->grad_requirs: True
-->name: emb_enc.enc.conv_layers.0.sep.weight -->grad_requirs: True
-->name: emb_enc.enc.conv_layers.0.sep.bias -->grad_requirs: True
-->name: emb_enc.enc.conv_layers.1.depth.weight -->grad_requirs: True
-->name: emb_enc.enc.conv_layers.1.sep.weight -->grad_requirs: True
-->name: emb_enc.en

In [18]:
device, _ = get_available_devices()

In [25]:
train_dataset = SQuAD('./data/train.npz', True)
train_loader = data.DataLoader(train_dataset,
                                   batch_size=24,
                                   shuffle=True,
                                   num_workers=1,
                                   collate_fn=collate_fn)

In [26]:
 for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:  
    cw_idxs = cw_idxs.to(device)
    cc_idxs = cc_idxs.to(device)
    qw_idxs = qw_idxs.to(device)
    qc_idxs = qc_idxs.to(device)
    batch_size = cw_idxs.size(0)
    log_p1, log_p2 = model(
                        cw_idxs, cc_idxs, qw_idxs,
                        qc_idxs) 
    break

In [28]:
from torch.autograd import Variable
x = Variable(torch.randn(1, 1), requires_grad=True)
with torch.autograd.profiler.profile() as prof:
    for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:  
        cw_idxs = cw_idxs.to(device)
        cc_idxs = cc_idxs.to(device)
        qw_idxs = qw_idxs.to(device)
        qc_idxs = qc_idxs.to(device)
        batch_size = cw_idxs.size(0)
        log_p1, log_p2 = model(
                            cw_idxs, cc_idxs, qw_idxs,
                            qc_idxs)
        break
    log_p1.backward()
# NOTE: some columns were removed for brevity
print(prof)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)